In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
with open('intents.json') as file:
    data = json.load(file)

In [ ]:
training_sentences = []
training_labels = []
labels = []
responses = []

In this all the tags are included in the label and 
training sentence=pattern
training label=tag
responses=responses

In [ ]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
num_classes = len(labels)
print(training_labels)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'about', 'name', 'name', 'name', 'fee', 'fee', 'fee', 'fee', 'faculty', 'faculty', 'faculty', 'faculty', 'faculty', 'faculty', 'Gyaan', 'Gyaan', 'Gyaan', 'Ragging', 'Achievments', 'departments', 'Academic calander', 'Syllabus', 'societies', 'Facilities', 'Photo galary', 'Placements', 'contact', 'Socials', 'Scholarships', 'NOTICES']


In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
print(training_labels)

[17 17 17 17 17 16 16 16 20 20 20 20 11 18 18 18 15 15 15 15 14 14 14 14
 14 14  3  3  3  7  1 13  0 10 19  2  5  6 12  9  8  4]


In [ ]:
print(training_sentences)

['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'Who are you?', 'what is your name', 'what should I call you', 'whats your name?', 'How to pay hostel fees', 'How to pay college fees', 'College fees structure', 'Hostel fees structure', 'who all are the facultly of MAIT', 'who all are the facultly of CSE', 'who all are the facultly of ECE', 'who all are the facultly of EEE', 'who all are the facultly of MAE', 'who all are the facultly of IT', 'what is the vision of mait', 'what is the mission of mait', 'what is the value of mait', 'What are steps take by mait against ragging', 'What are the achievements of mait', 'What all are the departments of MAIT', 'What is the academic calender of MAIT', 'What is the syllabus', 'What are the socities of MAIT', 'What are the sports facilities of MAIT', 'Some photos of MAIT', 'How are the placement of MAIT', 'How to contact mait authority', 'What are s

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
print(training_sentences)
print(sequences)

['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay', 'Bye', 'See you later', 'Goodbye', 'Thanks', 'Thank you', "That's helpful", 'Thanks for the help', 'Who are you?', 'what is your name', 'what should I call you', 'whats your name?', 'How to pay hostel fees', 'How to pay college fees', 'College fees structure', 'Hostel fees structure', 'who all are the facultly of MAIT', 'who all are the facultly of CSE', 'who all are the facultly of ECE', 'who all are the facultly of EEE', 'who all are the facultly of MAE', 'who all are the facultly of IT', 'what is the vision of mait', 'what is the mission of mait', 'what is the value of mait', 'What are steps take by mait against ragging', 'What are the achievements of mait', 'What all are the departments of MAIT', 'What is the academic calender of MAIT', 'What is the syllabus', 'What are the socities of MAIT', 'What are the sports facilities of MAIT', 'Some photos of MAIT', 'How are the placement of MAIT', 'How to contact mait authority', 'What are s

Need to learn about sequencial model from scratch(Not much understood from here)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
print(model)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 21)                357       
                                                                 
Total params: 16,901
Trainable params: 16,901
Non-trainable params: 0
____________________________________________________

In [ ]:
# to save the trained model
model.save("chat_model")

In [ ]:
import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print("User: ")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print("ChatBot:" + np.random.choice(i['responses']))


print("Start messaging with the bot (type quit to stop)!")
chat()

Start messaging with the bot (type quit to stop)!
User: 
1/1 [==============================] - 0s 83ms/step
ChatBot:You can call me MAIT_BUDDY
User: 
1/1 [==============================] - 0s 17ms/step
ChatBot:https://mait.ac.in/index.php/placements/about-placements follow here for placement details of MAIT
User: 
1/1 [==============================] - 0s 17ms/step
ChatBot:https://mait.ac.in/index.php/societies/technical Visit here for Technical socities
User: 
1/1 [==============================] - 0s 19ms/step
ChatBot:https://apsc.mait.ac.in/index.php/people/faculty visit here to know all the faculty of MAIT
User: 
1/1 [==============================] - 0s 17ms/step
ChatBot:https://apsc.mait.ac.in/index.php/people/faculty visit here to know all the faculty of MAIT
User: 
1/1 [==============================] - 0s 16ms/step
ChatBot:https://ece.mait.ac.in/index.php/people/faculty/faculty-profiles visit here to know all the faculty of cse
User: 
1/1 [==============================] - 0s